In [1]:
# 1) Imports e configuração básica
import numpy as np
from pathlib import Path
import pandas as pd

In [2]:
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

# 2) Caminhos do projeto
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / "data"

# 3) Leitura dos CSVs
results = pd.read_csv(DATA_RAW / "results.csv", parse_dates=["date"])
goalscorers = pd.read_csv(DATA_RAW / "goalscorers.csv", parse_dates=["date"])
shootouts = pd.read_csv(DATA_RAW / "shootouts.csv", parse_dates=["date"])

# 4) Checagens rápidas
print("results:", results.shape)
print("goalscorers:", goalscorers.shape)
print("shootouts:", shootouts.shape)

display(results.head(4))
display(goalscorers.head(4))
display(shootouts.head(4))


# 5) Validação mínima de colunas-chave (evita erro lá na frente)
# required_results = {"date", "home_team", "away_team", "home_score", "away_score", "neutral"}
# missing = required_results - set(results.columns)
# if missing:
# raise ValueError(f"results.csv está sem colunas esperadas: {missing}")

results: (44934, 9)
goalscorers: (41113, 8)
shootouts: (558, 4)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False


,date,home_team,away_team,team,scorer,minute,own_goal,penalty
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False


,date,home_team,away_team,winner
0,1967-08-22,India,Taiwan,Taiwan
1,1971-11-14,South Korea,Vietnam Republic,South Korea
2,1972-05-07,South Korea,Iraq,Iraq
3,1972-05-17,Thailand,South Korea,South Korea


In [3]:
wc = results.query("tournament == 'FIFA World Cup'").copy()

print("Jogos de Copa do Mundo:", wc.shape[0])
print("Copas:", wc["date"].dt.year.unique())


Jogos de Copa do Mundo: 964
Copas: [1930 1934 1938 1950 1954 1958 1962 1966 1970 1974 1978 1982 1986 1990
 1994 1998 2002 2006 2010 2014 2018 2022]


In [4]:
# Diferença absoluta de gols
wc["diff_gols"] = (wc["home_score"] - wc["away_score"]).abs()


In [5]:
wc["copa"] = wc["date"].dt.year
wc[["date", "copa"]].drop_duplicates().sort_values("copa").head(10)

,date,copa
1311,1930-07-13,1930
1313,1930-07-14,1930
1315,1930-07-15,1930
1316,1930-07-16,1930
1317,1930-07-17,1930
1320,1930-07-18,1930
1321,1930-07-19,1930
1323,1930-07-20,1930
1325,1930-07-21,1930
1326,1930-07-22,1930


In [6]:
baseline = (
    wc.groupby("copa")
      .agg(media_diff=("diff_gols", "mean"))
      .reset_index()
)

display(baseline.round(2))


,copa,media_diff
0,1930,2.67
1,1934,1.53
2,1938,2.11
3,1950,2.27
4,1954,3.00
5,1958,1.54
6,1962,1.53
7,1966,1.53
8,1970,1.53
9,1974,1.66


In [7]:
refino = (
    wc.groupby("copa")
      .agg(
          media_diff=("diff_gols", "mean"),
          mediana_diff=("diff_gols", "median")
      )
      .reset_index()
)

display(refino.round(2))


,copa,media_diff,mediana_diff
0,1930,2.67,3.0
1,1934,1.53,1.0
2,1938,2.11,2.0
3,1950,2.27,2.0
4,1954,3.00,2.0
5,1958,1.54,1.0
6,1962,1.53,1.0
7,1966,1.53,1.0
8,1970,1.53,1.0
9,1974,1.66,1.0


**Mediana = 1 → jogo típico equilibrado**

**Média caindo → menos jogos muito desequilibrados (goleadas)**

Dado que a mediana da diferença de gols se estabiliza em 1, a análise foi complementada com métricas de proporção para avaliar a distribuição dos níveis de equilíbrio das partidas.


In [8]:
# 1) Categorizar o equilíbrio por diferença de gols
wc["faixa_equilibrio"] = pd.cut(
    wc["diff_gols"],
    bins=[-0.1, 0, 1, 2, np.inf],
    labels=["empate", "1_gol", "2_gols", "3+_gols"]
)

# 2) Contagem por Copa e faixa
counts = (
    wc.groupby(["copa", "faixa_equilibrio"], observed=True)
      .size()
      .rename("jogos_faixa")
      .reset_index()
)

# 3) Percentual dentro de cada Copa
counts["pct"] = counts.groupby("copa")["jogos_faixa"].transform(lambda x: x / x.sum() * 100)

# 4) Pivot para formato final + fillna embutido (remove NaN) + total de jogos
tabela_final = (
    counts.pivot(index="copa", columns="faixa_equilibrio", values="pct")
          .fillna(0.0)   # <- AQUI resolve o NaN de uma vez
          .reset_index()
          .rename_axis(None, axis=1)
)

jogos_por_copa = wc.groupby("copa").size().rename("jogos").reset_index()
tabela_final = tabela_final.merge(jogos_por_copa, on="copa", how="left")

# 5) Ordenar colunas (garante presença de todas)
col_order = ["copa", "jogos", "empate", "1_gol", "2_gols", "3+_gols"]
for c in col_order:
    if c not in tabela_final.columns:
        tabela_final[c] = 0.0

tabela_final = tabela_final[col_order].sort_values("copa")

display(tabela_final.round(2))


,copa,jogos,empate,1_gol,2_gols,3+_gols
0,1930,18,0.00,27.78,16.67,55.56
1,1934,17,5.88,64.71,17.65,11.76
2,1938,18,16.67,27.78,33.33,22.22
3,1950,22,13.64,27.27,27.27,31.82
4,1954,26,7.69,19.23,30.77,42.31
5,1958,35,28.57,28.57,17.14,25.71
6,1962,32,15.62,37.50,37.50,9.38
7,1966,32,15.62,37.50,34.38,12.50
8,1970,32,15.62,43.75,15.62,25.00
9,1974,38,26.32,31.58,21.05,21.05


In [9]:
tabela_final = tabela_final.rename(columns={
    "empate": "% empates",
    "1_gol": "% vitórias por 1 gol",
    "2_gols": "% vitórias por 2 gols",
    "3+_gols": "% vitórias por 3+ gols"
})

display(tabela_final.round(2))


,copa,jogos,% empates,% vitórias por 1 gol,% vitórias por 2 gols,% vitórias por 3+ gols
0,1930,18,0.00,27.78,16.67,55.56
1,1934,17,5.88,64.71,17.65,11.76
2,1938,18,16.67,27.78,33.33,22.22
3,1950,22,13.64,27.27,27.27,31.82
4,1954,26,7.69,19.23,30.77,42.31
5,1958,35,28.57,28.57,17.14,25.71
6,1962,32,15.62,37.50,37.50,9.38
7,1966,32,15.62,37.50,34.38,12.50
8,1970,32,15.62,43.75,15.62,25.00
9,1974,38,26.32,31.58,21.05,21.05


In [10]:
tabela_print = tabela_final.copy()

# se quiser remover a coluna de jogos na exibição (opcional)
# tabela_print = tabela_print.drop(columns=["jogos"])

tabela_print["copa"] = tabela_print["copa"].astype("Int64")

cols_pct = ["% empates", "% vitórias por 1 gol", "% vitórias por 2 gols", "% vitórias por 3+ gols"]

fmt = {col: "{:.2f}%" for col in cols_pct}
fmt["copa"] = "{:d}"

(
    tabela_print.style
        .format(fmt)
        .background_gradient(subset=["% vitórias por 1 gol"], cmap="Greens")
        .background_gradient(subset=["% vitórias por 3+ gols"], cmap="Reds")
        .hide(axis="index")
)


copa,jogos,% empates,% vitórias por 1 gol,% vitórias por 2 gols,% vitórias por 3+ gols
1930,18,0.00%,27.78%,16.67%,55.56%
1934,17,5.88%,64.71%,17.65%,11.76%
1938,18,16.67%,27.78%,33.33%,22.22%
1950,22,13.64%,27.27%,27.27%,31.82%
1954,26,7.69%,19.23%,30.77%,42.31%
1958,35,28.57%,28.57%,17.14%,25.71%
1962,32,15.62%,37.50%,37.50%,9.38%
1966,32,15.62%,37.50%,34.38%,12.50%
1970,32,15.62%,43.75%,15.62%,25.00%
1974,38,26.32%,31.58%,21.05%,21.05%


In [11]:
# copas_chave = [1930, 1954, 1970, 1990, 1998, 2014, 2022]
tabela_print = tabela_final.copy()

tabela_print = tabela_print.drop(columns=["jogos"])
tabela_print["copa"] = tabela_print["copa"].astype("Int64")

col_verde = "% vitórias por 1 gol"
col_vermelho = "% vitórias por 3+ gols"

fmt = {col: "{:.2f}" for col in tabela_print.columns if col != "copa"}
fmt["copa"] = "{:d}"

(
    tabela_print.style
      .format(fmt)  # <- aqui resolve o .00 da copa
      .background_gradient(subset=[col_verde], cmap="Greens")
      .background_gradient(subset=[col_vermelho], cmap="Reds")
      .hide(axis="index")
)

copa,% empates,% vitórias por 1 gol,% vitórias por 2 gols,% vitórias por 3+ gols
1930,0.00,27.78,16.67,55.56
1934,5.88,64.71,17.65,11.76
1938,16.67,27.78,33.33,22.22
1950,13.64,27.27,27.27,31.82
1954,7.69,19.23,30.77,42.31
1958,28.57,28.57,17.14,25.71
1962,15.62,37.50,37.50,9.38
1966,15.62,37.50,34.38,12.50
1970,15.62,43.75,15.62,25.00
1974,26.32,31.58,21.05,21.05
